In [5]:
import pandas as pd

n_campi = 'CJBG'

levantamento = pd.read_excel('consolidacao/levantamento/2023_levantamento_CJBG.xlsx')
suap = pd.read_excel('consolidacao/suap/2023-08-01_14-36_CJBG_relatorio_almox.xlsx')

# removendo itens nulos e deixando apenas os retificados
retificacao = levantamento[levantamento['Retificação'].str.contains('Retificação').fillna(False)]
# lista de itens vencidos
# vencido = levantamento[levantamento['Material Vencido - MV'].str.contains('x').fillna(False)]
# lista de itens não identificados
# nao_identificado = levantamento[levantamento['Sem identificação - SI'].str.contains('x').fillna(False)]

print(levantamento.info())
print(suap.info())

# planilha contendo apenas itens válidos
levantamento_ret = levantamento.drop_duplicates(subset=['Código', 'Retificação'], keep='last')
levantamento_ret = levantamento_ret.rename(columns={'Código': 'codigo'})
# elemento despesa do levantamento
ed_lev = levantamento_ret['codigo'].str[3:5]


# adicionando ed ao data frame geral
# levantamento_ret['ED'] = ed_lev

# lista de servidores
servidores = levantamento['Endereço de e-mail'].drop_duplicates(keep='last')
# reindexando uutilizando list comprehension
servidores.index = [x for x in range(0, len(servidores))]

levantamento_cross = levantamento_ret.drop(columns=['Material de pouca movimentação - PM', 'Material Vencido - MV',
                                                 'Material próximo ao vencimento - PV', 'Material danificado ou avariado - MD',
                                                 'Material Obsoleto - MO', 'Sem identificação - SI', 'Valor unitário',
                                                 'Campus', 'Retificação', 'Carimbo de data/hora'])
levantamento_cross = levantamento_cross.rename(columns={'Endereço de e-mail': 'servidor', 'Nome': 'descricao_lev',
                                                        'Quantidade': 'qtd_lev'})
levantamento_cross = levantamento_cross[['codigo', 'descricao_lev', 'qtd_lev', 'servidor']]

suap_cross = suap.rename(columns={'descricao': 'descricao_suap', 'qtd': 'qtd_suap'})
suap_cross = suap_cross.drop(columns=['vlr_total', 'vlr_unt'])
display(suap_cross)
display(levantamento_cross)

consolidacao = pd.merge(suap_cross, levantamento_cross, on='codigo', how='outer', indicator=True)
consolidacao.to_excel(f'consolidacao/{n_campi}_consolidacao.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   Carimbo de data/hora                  225 non-null    datetime64[ns]
 1   Endereço de e-mail                    225 non-null    object        
 2   Retificação                           3 non-null      object        
 3   Código                                225 non-null    object        
 4   Nome                                  223 non-null    object        
 5   Quantidade                            225 non-null    int64         
 6   Campus                                86 non-null     object        
 7   Localização                           41 non-null     object        
 8   Valor unitário                        37 non-null     float64       
 9   Sem identificação - SI                0 non-null      float64       
 10  Ma

,codigo,descricao_suap,und,qtd_suap,ed
0,30.07.847,"CAFÉ, APRESENTAÇÃO TORRADO E MOÍDO, CARACTERÍS...",UN,17,07 - GENEROS DE ALIMENTACAO
1,30.16.1026,"GRAMPEADOR DE METAL, TIPO PROFISSIONAL CAPACID...",UN,5,16 - MATERIAL DE EXPEDIENTE
2,30.16.1093,"BLOCO AUTO-COLANTE PARA RECADO, COR NATURAL, D...",UN,4,16 - MATERIAL DE EXPEDIENTE
3,30.16.1096,"COLCHETE (BAILARINA), Nº 15, COMPRIMENTO DA HA...",CAIXA,4,16 - MATERIAL DE EXPEDIENTE
4,30.16.1144,"PASTA SUSPENSA, MATERIAL CARTÃO MARMORIZADO PL...",UN,1364,16 - MATERIAL DE EXPEDIENTE
...,...,...,...,...,...
222,30.42.1160,"TORNEIRA, Material corpo metal cromado, tipo l...",UN,29,42 - FERRAMENTAS
223,30.44.41,Círculo ou seta indicativo de H2O. Cor interna...,UN,5,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
224,30.44.42,Círculo ou seta indicativo de CO2. Cor interna...,UN,16,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS
225,30.44.43,Círculo ou seta indicativo de PQS. Cor interna...,UN,2,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS


,codigo,descricao_lev,qtd_lev,servidor
0,30.16.2565,"PASTA ARQUIVO, MATERIAL CART",56,cmpsg@jaboatao.ifpe.edu.br
1,30.16.2551,CANETA ESFEROGR,13,cmpsg@jaboatao.ifpe.edu.br
2,30.16.2554,"MINA GRAFITE, MATERIAL GRAFITA, DI",6,cmpsg@jaboatao.ifpe.edu.br
3,30.16.2560,"MINA GRAFITE, MATERIAL GRAFITA, DI",7,cmpsg@jaboatao.ifpe.edu.br
4,30.16.2057,"TINTA PARA CARIMBO, COR PRETA, COMPONENTES BASE D",10,cmpsg@jaboatao.ifpe.edu.br
...,...,...,...,...
220,30.16.1296,Papel A3 90g,7,cmpsg@jaboatao.ifpe.edu.br
221,30.17.945,CABO TELEF,2,cmpsg@jaboatao.ifpe.edu.br
222,30.17.965,"TECLADO MICROCOMPUTADOR, TIPO NUM",2,cmpsg@jaboatao.ifpe.edu.br
223,30.17.571,"CABO EXTENSOR, FLEX",31,cmpsg@jaboatao.ifpe.edu.br


In [ ]:
consolidacao['diferenca'] = consolidacao['qtd_suap'] - consolidacao['qtd_lev']
display(consolidacao)

pedidos = consolidacao[consolidacao['_merge'].str.contains('both')]
pedidos = pedidos[['codigo','ed' ,'descricao_suap', 'qtd_suap', 'qtd_lev', 'diferenca']]

acao = []
for valor in pedidos['diferenca']:
    
    if valor > 0:
        baixa = "pedido no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "saida sem pedido"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

pedidos['acao'] = acao
pedidos.to_excel(f'consolidacao/pedidos/{n_campi}_pedidos.xlsx', index=False)
display(pedidos)